# Analitika Media Sosial #

## Tugas Preprocessing data ##

* Agung Kurniawan
* A12.2020.06364

## Proses mengambil data dari twiter

## Load library ##

In [1]:
import pandas as pd
import time
import tweepy
import numpy as np

## Otentifikasi dan hak akses ##

In [2]:
def twitter_config():
    """
    Fungsi utilitas untuk mengkonfigurasi konsumsi file API Twitter dengan␣
    ,!kunci yang disediakan.
    """
    # Otentikasi dan akses menggunakan kunci:
    auth = tweepy.OAuthHandler("TZdvNUurG6MSIzxuYTlZMOM72",
    "eQxZiDCi978SEx12Q7CBJ1l65hivaxUyPCdQZ2U1zNdQbHz6TU")
    auth.set_access_token("1572477411916402688-bnFJjEFGqjC6PmCOo55zeTPWwmV5pQ",
    "tz8xMrwG137fU3QSOGCYP24cJSBhTQJzYbfvjxMMqoDbJ")
    # Kembalikan akses ke API:
    api = tweepy.API(auth)
    try:
        api.verify_credentials()
        print("Authentication OK")
    except:
        print("Error during authentication")
    return api

In [3]:
# buat extractor object
extractor = twitter_config()

Authentication OK


## Masukan Username ##

In [4]:
tweets = extractor.user_timeline(screen_name="yurayunita", count=15)
print("Tweets terambil: {}.\n".format(len(tweets)))


print("5 tweet teratas:\n")
for tweet in tweets[:5]:
    print(tweet.text)
    print()

Tweets terambil: 15.

5 tweet teratas:

Dan hari ini juga adalah perayaan untuk kamu yang terus mendengarkan album ‘Tutur Batin’, terima kasih telah member… https://t.co/GTmzVXxKFw

Jaga umur karya untuk terus selalu diingat, adalah sebuah usaha yang tidak mungkin dilakukan dari penciptanya saja,… https://t.co/WxqqvAC1GE

Hapus riasan di atas panggung, merayakan momen 1 tahun rilisnya album ‘Tutur Batin’. 

Momen ini penting bagiku unt… https://t.co/ZikFDlM55j

Selamat ulang tahun, Tutur Batin 🤍 https://t.co/P42DjWLbhs

Bukan hal yang mudah. Tapi aku ingin terus belajar bahwa mengejar kesempurnaan itu ga akan ada habisnya sampai aku… https://t.co/T2iLXio4Ma



## Masuk ke data frame Pandas ##

In [5]:
# Kita dapat membuat kerangka data sebagai berikut:
dataset = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])

# Kami membuat tampilan kerangka data:
display(dataset.head(10))

,Tweets
0,Dan hari ini juga adalah perayaan untuk kamu y...
1,"Jaga umur karya untuk terus selalu diingat, ad..."
2,"Hapus riasan di atas panggung, merayakan momen..."
3,"Selamat ulang tahun, Tutur Batin 🤍 https://t.c..."
4,Bukan hal yang mudah. Tapi aku ingin terus bel...
5,Satu tahun yang lalu aku merilis album Tutur B...
6,Kisah perjalananku hari ini. Semoga lancar &am...
7,RT @HipHipYURA: SCHEDULE YURA YUNITA\nOktober ...
8,Akan kuhadapi semuanya walau penuh dengan plot...
9,Jika diberi kesempatan kembali ke masa lalu da...


In [6]:
dataset.to_csv('dataset_yura.csv',sep=',')

## Preprocessing data

## Load library ##

In [11]:
import re
import string
import time
from copy import deepcopy

## Load Dataset ##

In [12]:
import pandas as pd

In [15]:
df = pd.read_csv('dataset_narasi_fixlabel.csv',sep=';')

In [16]:
df.head()

,Column1,Unnamed: 0,Tweets,Label
0,0,0,Ketua DPP PHRI DKI Jakarta Sutrisno Iwantono b...,1
1,1,1,"Soalnya, RKUHP yang baru berisi pasal perzinah...",1
2,2,2,"#Semuabisakena. Inget, kan, sama kampanye peno...",1
3,3,3,"Gugatannya udah kalah. Selanjutnya, Pak Jokowi...",1
4,4,4,Semoga aja amunisi Kominfo bisa efektif buat n...,0


In [17]:
df

,Column1,Unnamed: 0,Tweets,Label
0,0,0,Ketua DPP PHRI DKI Jakarta Sutrisno Iwantono b...,1
1,1,1,"Soalnya, RKUHP yang baru berisi pasal perzinah...",1
2,2,2,"#Semuabisakena. Inget, kan, sama kampanye peno...",1
3,3,3,"Gugatannya udah kalah. Selanjutnya, Pak Jokowi...",1
4,4,4,Semoga aja amunisi Kominfo bisa efektif buat n...,0
...,...,...,...,...
95,95,95,Kenapa sih maksa banget jadwal pertandingan di...,1
96,96,96,Kelima terdakwa dalam kasus pembunuhan Yosua t...,1
97,97,97,Lembaga Perlindungan Saksi dan Korban (LPSK) m...,1
98,98,98,"Eliezer beserta Ferdy Sambo, Putri Candrawathi...",1


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  199 non-null    int64 
 1   Tweets      199 non-null    object
 2   Source      199 non-null    object
 3   Likes       199 non-null    int64 
 4   RTs         199 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 7.9+ KB


## Preprocessing Dataset ##

In [14]:
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['email', 'percent', 'money', 'phone', 'user',
        'time', 'date', 'number'],
    # terms that will be annotated
    #annotate={"hashtag", "allcaps", "elongated", "repeated",'emphasis', 'censored'},
    annotate={"hashtag"},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)


C:\Users\HP\anaconda3\lib\site-packages\ekphrasis\classes\tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading twitter - 1grams ...


C:\Users\HP\anaconda3\lib\site-packages\ekphrasis\classes\exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [15]:
def bersih_data(text):
    return " ".join(text_processor.pre_process_doc(text))

In [16]:
def non_ascii(text):
    return text.encode('ascii', 'replace').decode('ascii')

def remove_space_alzami(text):
    return " ".join(text.split())

def remove_emoji_alzami(text):
    return ' '.join(re.sub("([x#][A-Za-z0-9]+)"," ", text).split())

def remove_tab(text):
    return text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")

def remove_tab2(text):
    return re.sub('\s+',' ',text)

def remove_rt(text):
    return text.replace('RT'," ")

def remove_mention(text):
    return ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())

def remove_incomplete_url(text):
    return text.replace("http://", " ").replace("https://", " ")

def remove_single_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

def remove_excessive_dot(text):
    return text.replace('..'," ")

def change_stripe(text):
    return text.replace('-'," ")

def lower(text):
    return text.lower()

def remove_single_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

def remove_excessive_dot(text):
    return text.replace('..'," ")

def lower(text):
    return text.lower()

def remove_whitespace_LT(text):
    return text.strip()

def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

def remove_punctuation(text):
    remove = string.punctuation
    remove = remove.replace("_", "") # don't remove hyphens
    pattern = r"[{}]".format(remove) # create the pattern
    return re.sub(pattern, "", text) 

In [17]:
def remove_number_eks(text):
    return text.replace('<number>'," ")

def remove_angka(text):
    return re.sub(r"\d+", "", text) 

def remove_URL_eks(text):
    return text.replace('URL'," ").replace('url'," ")

def space_punctuation(text):
    return re.sub('(?<! )(?=[.,!?()])|(?<=[.,!?()])(?! )', r' ', text)

## Melakukan Pembersihan dengan memanggil fungsi diatas ##

In [18]:
i = 0
final_string = []
s = ""
for text in df['Tweets'].values:
    filteredSentence = []
    EachReviewText = ""
    proc = remove_rt(text)
    proc = lower(proc)
    proc = change_stripe(proc)
    proc = remove_emoji_alzami(proc)
    proc = remove_tab(proc)
    proc = remove_tab2(proc)
    proc = non_ascii(proc)
    proc = remove_incomplete_url(proc)
    proc = remove_excessive_dot(proc)
    proc = remove_whitespace_LT(proc)
    proc = remove_whitespace_multiple(proc)
    proc = remove_single_char(proc)
    proc = space_punctuation(proc)
    proc = remove_punctuation(proc)
    proc = remove_space_alzami(proc)
    proc = bersih_data(proc)
    proc = remove_number_eks(proc)
    proc = remove_angka(proc) 
    proc = remove_URL_eks(proc)
    EachReviewText = proc
    final_string.append(EachReviewText)

In [19]:
df["step1"] = final_string

In [20]:
df.head(10)

,Unnamed: 0,Tweets,Source,Likes,RTs,step1
0,0,"Sejauh mana kamu mencari, ga harus ada jawaban...",Twitter for iPhone,9473,3740,sejauh mana kamu mencari ga harus ada jawabann...
1,1,Jadwalku bulan ini untuk bertemu kamu ☺️ https...,Twitter for iPhone,597,56,jadwalku bulan ini untuk bertemu kamu cosr
2,2,@iffhzzi @anjwiiink Desember aku ke Medan! Tun...,Twitter for iPhone,8,2,iffhzzi anjwiiink desember aku ke medan tunggu...
3,3,@sunshineinevryd Besok aku ke Samarinda,Twitter for iPhone,32,0,sunshineinevryd besok aku ke samarinda
4,4,Kenapa udah lanjut lagi ke yang lain? Memangny...,Twitter for iPhone,19783,3182,kenapa udah lanjut lagi ke yang lain memangnya...
5,5,"RT @jowiwi26: Teh @yurayunita, more like yura ...",Twitter for iPhone,0,5,jowiwi teh yurayunita more like yura yunari co...
6,6,😭😭😭 https://t.co/DjHi7kwew3,Twitter for iPhone,796,82,codjhikwew
7,7,Alasan termanis apa yang pernah kamu denger? P...,Twitter for iPhone,14474,1328,alasan termanis apa yang pernah kamu denger pa...
8,8,@WalterWisnu @bb_festival takut penontonnya ce...,Twitter for iPhone,5,0,walterwisnu bb_festival takut penontonnya cena...
9,9,Haha seru amat deh konsep acara semalam! Next ...,Twitter for iPhone,81,3,haha seru amat deh konsep acara semalam ne kit...


## Hapus data kosong

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  199 non-null    int64 
 1   Tweets      199 non-null    object
 2   Source      199 non-null    object
 3   Likes       199 non-null    int64 
 4   RTs         199 non-null    int64 
 5   step1       199 non-null    object
dtypes: int64(3), object(3)
memory usage: 9.5+ KB


In [22]:
df_hapus = df[~df['step1'].str.contains(" ")]

In [23]:
df_hapus.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11 entries, 6 to 185
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  11 non-null     int64 
 1   Tweets      11 non-null     object
 2   Source      11 non-null     object
 3   Likes       11 non-null     int64 
 4   RTs         11 non-null     int64 
 5   step1       11 non-null     object
dtypes: int64(3), object(3)
memory usage: 616.0+ bytes


In [24]:
df_hapus.head(10)

,Unnamed: 0,Tweets,Source,Likes,RTs,step1
6,6,😭😭😭 https://t.co/DjHi7kwew3,Twitter for iPhone,796,82,codjhikwew
20,20,https://t.co/0KQDCTOKns,Twitter for iPhone,150,18,cokqdctokns
62,62,https://t.co/qKGOm5Twyj,Twitter for iPhone,102,4,coqkgomtwyj
70,70,🥹☺️☺️🤗🤗 https://t.co/cuDwQhgdcO,Twitter for iPhone,275,16,cocudwqhgdco
105,105,@_bernacleboy 😭😭😭,Twitter for iPhone,70,1,_bernacleboy
115,115,@cursedkidd y,Twitter for iPhone,10778,346,cursedkidd
117,117,😭😭😭 https://t.co/mXT1OSEDBl,Twitter for iPhone,2682,120,co
162,162,😘😘😘 https://t.co/acyqbMKG3E,Twitter for iPhone,215,22,coacyqbmkge
168,168,@convomfs 😭😭😭,Twitter for iPhone,365,15,convomfs
184,184,Monang,Twitter for iPhone,315,60,monang


In [25]:
df_new = df[~df.isin(df_hapus)].dropna()

In [26]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 188 entries, 0 to 198
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  188 non-null    float64
 1   Tweets      188 non-null    object 
 2   Source      188 non-null    object 
 3   Likes       188 non-null    float64
 4   RTs         188 non-null    float64
 5   step1       188 non-null    object 
dtypes: float64(3), object(3)
memory usage: 10.3+ KB


In [27]:
df_new

,Unnamed: 0,Tweets,Source,Likes,RTs,step1
0,0.0,"Sejauh mana kamu mencari, ga harus ada jawaban...",Twitter for iPhone,9473.0,3740.0,sejauh mana kamu mencari ga harus ada jawabann...
1,1.0,Jadwalku bulan ini untuk bertemu kamu ☺️ https...,Twitter for iPhone,597.0,56.0,jadwalku bulan ini untuk bertemu kamu cosr
2,2.0,@iffhzzi @anjwiiink Desember aku ke Medan! Tun...,Twitter for iPhone,8.0,2.0,iffhzzi anjwiiink desember aku ke medan tunggu...
3,3.0,@sunshineinevryd Besok aku ke Samarinda,Twitter for iPhone,32.0,0.0,sunshineinevryd besok aku ke samarinda
4,4.0,Kenapa udah lanjut lagi ke yang lain? Memangny...,Twitter for iPhone,19783.0,3182.0,kenapa udah lanjut lagi ke yang lain memangnya...
...,...,...,...,...,...,...
194,194.0,Melepaskan itu sulit ya. Tapi ga bisa jadi dir...,Twitter for iPhone,26611.0,6925.0,melepaskan itu sulit ya tapi ga bisa jadi diri...
195,195.0,Tuhan maha tau yang terbaik buat aku. Selalu j...,Twitter for iPhone,20915.0,8091.0,tuhan maha tau yang terbaik buat aku selalu ja...
196,196.0,Rasa tenang mungkin belum juga kunjung datang ...,Twitter Media Studio,66.0,6.0,rasa tenang mungkin belum juga kunjung datang ...
197,197.0,"Kebenaran akan terdengar oleh telinga batinmu,...",Twitter for iPhone,12905.0,3691.0,kebenaran akan terdengar oleh telinga batinmu ...


## Normalisasi kata

In [28]:
# token
import nltk
nltk.download('punkt')m
from nltk.tokenize import word_tokenize 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [29]:
def word_tokenize_wrapper(text):
  return word_tokenize(text)

In [31]:
df_new['tokens'] = df['step1'].apply(word_tokenize_wrapper)

In [32]:
df_new.head(10)

,Unnamed: 0,Tweets,Source,Likes,RTs,step1,tokens
0,0.0,"Sejauh mana kamu mencari, ga harus ada jawaban...",Twitter for iPhone,9473.0,3740.0,sejauh mana kamu mencari ga harus ada jawabann...,"[sejauh, mana, kamu, mencari, ga, harus, ada, ..."
1,1.0,Jadwalku bulan ini untuk bertemu kamu ☺️ https...,Twitter for iPhone,597.0,56.0,jadwalku bulan ini untuk bertemu kamu cosr,"[jadwalku, bulan, ini, untuk, bertemu, kamu, c..."
2,2.0,@iffhzzi @anjwiiink Desember aku ke Medan! Tun...,Twitter for iPhone,8.0,2.0,iffhzzi anjwiiink desember aku ke medan tunggu...,"[iffhzzi, anjwiiink, desember, aku, ke, medan,..."
3,3.0,@sunshineinevryd Besok aku ke Samarinda,Twitter for iPhone,32.0,0.0,sunshineinevryd besok aku ke samarinda,"[sunshineinevryd, besok, aku, ke, samarinda]"
4,4.0,Kenapa udah lanjut lagi ke yang lain? Memangny...,Twitter for iPhone,19783.0,3182.0,kenapa udah lanjut lagi ke yang lain memangnya...,"[kenapa, udah, lanjut, lagi, ke, yang, lain, m..."
5,5.0,"RT @jowiwi26: Teh @yurayunita, more like yura ...",Twitter for iPhone,0.0,5.0,jowiwi teh yurayunita more like yura yunari co...,"[jowiwi, teh, yurayunita, more, like, yura, yu..."
7,7.0,Alasan termanis apa yang pernah kamu denger? P...,Twitter for iPhone,14474.0,1328.0,alasan termanis apa yang pernah kamu denger pa...,"[alasan, termanis, apa, yang, pernah, kamu, de..."
8,8.0,@WalterWisnu @bb_festival takut penontonnya ce...,Twitter for iPhone,5.0,0.0,walterwisnu bb_festival takut penontonnya cena...,"[walterwisnu, bb_festival, takut, penontonnya,..."
9,9.0,Haha seru amat deh konsep acara semalam! Next ...,Twitter for iPhone,81.0,3.0,haha seru amat deh konsep acara semalam ne kit...,"[haha, seru, amat, deh, konsep, acara, semalam..."
10,10.0,No sporty No Party! Begitu menurut konsep acar...,Twitter Media Studio,322.0,23.0,no sporty no party begitu menurut konsep acara...,"[no, sporty, no, party, begitu, menurut, konse..."


In [34]:
normalized_word = pd.read_csv('kamus_clean.csv')

In [35]:
normalized_word_dict = {}

for index, row in normalized_word.iterrows():
    if row[0] not in normalized_word_dict:
        normalized_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalized_word_dict[term] if term in normalized_word_dict else term for term in document]

In [36]:
df_new['final_tokens'] = df_new['tokens'].apply(normalized_term)

In [37]:
i=0
final_string_tokens = []
for text in df_new['final_tokens'].values:
    EachReviewText = ""
    EachReviewText = ' '.join(text)
    final_string_tokens.append(EachReviewText)

In [38]:
df_new["step2"] = final_string_tokens

In [39]:
df_new.head(10)

,Unnamed: 0,Tweets,Source,Likes,RTs,step1,tokens,final_tokens,step2
0,0.0,"Sejauh mana kamu mencari, ga harus ada jawaban...",Twitter for iPhone,9473.0,3740.0,sejauh mana kamu mencari ga harus ada jawabann...,"[sejauh, mana, kamu, mencari, ga, harus, ada, ...","[sejauh, mana, kamu, mencari, ga, harus, ada, ...",sejauh mana kamu mencari ga harus ada jawabann...
1,1.0,Jadwalku bulan ini untuk bertemu kamu ☺️ https...,Twitter for iPhone,597.0,56.0,jadwalku bulan ini untuk bertemu kamu cosr,"[jadwalku, bulan, ini, untuk, bertemu, kamu, c...","[jadwalku, bulan, ini, untuk, bertemu, kamu, c...",jadwalku bulan ini untuk bertemu kamu cosr
2,2.0,@iffhzzi @anjwiiink Desember aku ke Medan! Tun...,Twitter for iPhone,8.0,2.0,iffhzzi anjwiiink desember aku ke medan tunggu...,"[iffhzzi, anjwiiink, desember, aku, ke, medan,...","[iffhzzi, anjwiiink, desember, aku, ke, medan,...",iffhzzi anjwiiink desember aku ke medan tunggu...
3,3.0,@sunshineinevryd Besok aku ke Samarinda,Twitter for iPhone,32.0,0.0,sunshineinevryd besok aku ke samarinda,"[sunshineinevryd, besok, aku, ke, samarinda]","[sunshineinevryd, besok, aku, ke, samarinda]",sunshineinevryd besok aku ke samarinda
4,4.0,Kenapa udah lanjut lagi ke yang lain? Memangny...,Twitter for iPhone,19783.0,3182.0,kenapa udah lanjut lagi ke yang lain memangnya...,"[kenapa, udah, lanjut, lagi, ke, yang, lain, m...","[kenapa, udah, lanjut, lagi, ke, yang, lain, m...",kenapa udah lanjut lagi ke yang lain memangnya...
5,5.0,"RT @jowiwi26: Teh @yurayunita, more like yura ...",Twitter for iPhone,0.0,5.0,jowiwi teh yurayunita more like yura yunari co...,"[jowiwi, teh, yurayunita, more, like, yura, yu...","[jowiwi, teh, yurayunita, more, like, yura, yu...",jowiwi teh yurayunita more like yura yunari co...
7,7.0,Alasan termanis apa yang pernah kamu denger? P...,Twitter for iPhone,14474.0,1328.0,alasan termanis apa yang pernah kamu denger pa...,"[alasan, termanis, apa, yang, pernah, kamu, de...","[alasan, termanis, apa, yang, pernah, kamu, de...",alasan termanis apa yang pernah kamu denger pa...
8,8.0,@WalterWisnu @bb_festival takut penontonnya ce...,Twitter for iPhone,5.0,0.0,walterwisnu bb_festival takut penontonnya cena...,"[walterwisnu, bb_festival, takut, penontonnya,...","[walterwisnu, bb_festival, takut, penontonnya,...",walterwisnu bb_festival takut penontonnya cena...
9,9.0,Haha seru amat deh konsep acara semalam! Next ...,Twitter for iPhone,81.0,3.0,haha seru amat deh konsep acara semalam ne kit...,"[haha, seru, amat, deh, konsep, acara, semalam...","[haha, seru, amat, deh, konsep, acara, semalam...",haha seru amat deh konsep acara semalam ne kit...
10,10.0,No sporty No Party! Begitu menurut konsep acar...,Twitter Media Studio,322.0,23.0,no sporty no party begitu menurut konsep acara...,"[no, sporty, no, party, begitu, menurut, konse...","[no, sporty, no, party, begitu, menurut, konse...",no sporty no party begitu menurut konsep acara...


In [40]:
df_new.to_csv('clean_dataset_yura.csv',sep=",")

## Proses stemming dengan Sastrawi

## Load dataset

In [41]:
import pandas as pd

In [42]:
dataset = pd.read_csv('clean_dataset_yura.csv', sep=',')

In [43]:
dataset.head()

,Unnamed: 0.1,Unnamed: 0,Tweets,Source,Likes,RTs,step1,tokens,final_tokens,step2
0,0,0.0,"Sejauh mana kamu mencari, ga harus ada jawaban...",Twitter for iPhone,9473.0,3740.0,sejauh mana kamu mencari ga harus ada jawabann...,"['sejauh', 'mana', 'kamu', 'mencari', 'ga', 'h...","['sejauh', 'mana', 'kamu', 'mencari', 'ga', 'h...",sejauh mana kamu mencari ga harus ada jawabann...
1,1,1.0,Jadwalku bulan ini untuk bertemu kamu ☺️ https...,Twitter for iPhone,597.0,56.0,jadwalku bulan ini untuk bertemu kamu cosr,"['jadwalku', 'bulan', 'ini', 'untuk', 'bertemu...","['jadwalku', 'bulan', 'ini', 'untuk', 'bertemu...",jadwalku bulan ini untuk bertemu kamu cosr
2,2,2.0,@iffhzzi @anjwiiink Desember aku ke Medan! Tun...,Twitter for iPhone,8.0,2.0,iffhzzi anjwiiink desember aku ke medan tunggu...,"['iffhzzi', 'anjwiiink', 'desember', 'aku', 'k...","['iffhzzi', 'anjwiiink', 'desember', 'aku', 'k...",iffhzzi anjwiiink desember aku ke medan tunggu...
3,3,3.0,@sunshineinevryd Besok aku ke Samarinda,Twitter for iPhone,32.0,0.0,sunshineinevryd besok aku ke samarinda,"['sunshineinevryd', 'besok', 'aku', 'ke', 'sam...","['sunshineinevryd', 'besok', 'aku', 'ke', 'sam...",sunshineinevryd besok aku ke samarinda
4,4,4.0,Kenapa udah lanjut lagi ke yang lain? Memangny...,Twitter for iPhone,19783.0,3182.0,kenapa udah lanjut lagi ke yang lain memangnya...,"['kenapa', 'udah', 'lanjut', 'lagi', 'ke', 'ya...","['kenapa', 'udah', 'lanjut', 'lagi', 'ke', 'ya...",kenapa udah lanjut lagi ke yang lain memangnya...


## Load sastrawi sebagai stemmer

In [44]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [45]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
 
factory = StopWordRemoverFactory()

more_stopword = ['sih', 'nya','rt','loh','lah', 'dd', 'mah', 'nye', 'eh', 'ehh', 'ah', 'yang','yg']
 
# Tambahkan Stopword Baru
data = factory.get_stop_words()+more_stopword

stopwords_sastrawi = factory.create_stop_word_remover()

dictionary = ArrayDictionary(data)
str_stopwords = StopWordRemover(dictionary)

In [46]:
dataset['finalText_str'] = dataset['step2'].apply(str)

In [47]:
i=0
final_string = []
s = ""
for sentence in dataset["finalText_str"].values:
    filteredSentence = []
    EachReviewText = ""
    st = str_stopwords.remove(sentence)
    s = (stemmer.stem(st))
    filteredSentence.append(s)
    
    EachReviewText = ' '.join(filteredSentence)
    final_string.append(EachReviewText)

In [48]:
dataset.loc[:, ('ProcessedText')] = final_string

In [49]:
dataset

,Unnamed: 0.1,Unnamed: 0,Tweets,Source,Likes,RTs,step1,tokens,final_tokens,step2,finalText_str,ProcessedText
0,0,0.0,"Sejauh mana kamu mencari, ga harus ada jawaban...",Twitter for iPhone,9473.0,3740.0,sejauh mana kamu mencari ga harus ada jawabann...,"['sejauh', 'mana', 'kamu', 'mencari', 'ga', 'h...","['sejauh', 'mana', 'kamu', 'mencari', 'ga', 'h...",sejauh mana kamu mencari ga harus ada jawabann...,sejauh mana kamu mencari ga harus ada jawabann...,jauh mana kamu cari ga ada jawab sekarang saba...
1,1,1.0,Jadwalku bulan ini untuk bertemu kamu ☺️ https...,Twitter for iPhone,597.0,56.0,jadwalku bulan ini untuk bertemu kamu cosr,"['jadwalku', 'bulan', 'ini', 'untuk', 'bertemu...","['jadwalku', 'bulan', 'ini', 'untuk', 'bertemu...",jadwalku bulan ini untuk bertemu kamu cosr,jadwalku bulan ini untuk bertemu kamu cosr,jadwal bulan untuk temu kamu cosr
2,2,2.0,@iffhzzi @anjwiiink Desember aku ke Medan! Tun...,Twitter for iPhone,8.0,2.0,iffhzzi anjwiiink desember aku ke medan tunggu...,"['iffhzzi', 'anjwiiink', 'desember', 'aku', 'k...","['iffhzzi', 'anjwiiink', 'desember', 'aku', 'k...",iffhzzi anjwiiink desember aku ke medan tunggu...,iffhzzi anjwiiink desember aku ke medan tunggu...,iffhzzi anjwiiink desember aku medan tungguin ...
3,3,3.0,@sunshineinevryd Besok aku ke Samarinda,Twitter for iPhone,32.0,0.0,sunshineinevryd besok aku ke samarinda,"['sunshineinevryd', 'besok', 'aku', 'ke', 'sam...","['sunshineinevryd', 'besok', 'aku', 'ke', 'sam...",sunshineinevryd besok aku ke samarinda,sunshineinevryd besok aku ke samarinda,sunshineinevryd besok aku samarinda
4,4,4.0,Kenapa udah lanjut lagi ke yang lain? Memangny...,Twitter for iPhone,19783.0,3182.0,kenapa udah lanjut lagi ke yang lain memangnya...,"['kenapa', 'udah', 'lanjut', 'lagi', 'ke', 'ya...","['kenapa', 'udah', 'lanjut', 'lagi', 'ke', 'ya...",kenapa udah lanjut lagi ke yang lain memangnya...,kenapa udah lanjut lagi ke yang lain memangnya...,udah lanjut ke lain memang lalu selesai
...,...,...,...,...,...,...,...,...,...,...,...,...
183,194,194.0,Melepaskan itu sulit ya. Tapi ga bisa jadi dir...,Twitter for iPhone,26611.0,6925.0,melepaskan itu sulit ya tapi ga bisa jadi diri...,"['melepaskan', 'itu', 'sulit', 'ya', 'tapi', '...","['melepaskan', 'itu', 'sulit', 'ya', 'tapi', '...",melepaskan itu sulit ya tapi ga bisa jadi diri...,melepaskan itu sulit ya tapi ga bisa jadi diri...,lepas sulit tapi ga jadi diri sendiri sama pun...
184,195,195.0,Tuhan maha tau yang terbaik buat aku. Selalu j...,Twitter for iPhone,20915.0,8091.0,tuhan maha tau yang terbaik buat aku selalu ja...,"['tuhan', 'maha', 'tau', 'yang', 'terbaik', 'b...","['tuhan', 'maha', 'tau', 'yang', 'terbaik', 'b...",tuhan maha tau yang terbaik buat aku selalu ja...,tuhan maha tau yang terbaik buat aku selalu ja...,tuhan maha tau baik buat aku selalu jadi ingat...
185,196,196.0,Rasa tenang mungkin belum juga kunjung datang ...,Twitter Media Studio,66.0,6.0,rasa tenang mungkin belum juga kunjung datang ...,"['rasa', 'tenang', 'mungkin', 'belum', 'juga',...","['rasa', 'tenang', 'mungkin', 'belum', 'juga',...",rasa tenang mungkin belum juga kunjung datang ...,rasa tenang mungkin belum juga kunjung datang ...,rasa tenang mungkin juga kunjung datang lagu l...
186,197,197.0,"Kebenaran akan terdengar oleh telinga batinmu,...",Twitter for iPhone,12905.0,3691.0,kebenaran akan terdengar oleh telinga batinmu ...,"['kebenaran', 'akan', 'terdengar', 'oleh', 'te...","['kebenaran', 'akan', 'terdengar', 'oleh', 'te...",kebenaran akan terdengar oleh telinga batinmu ...,kebenaran akan terdengar oleh telinga batinmu ...,benar dengar telinga batin lihat mata batin as...


In [51]:
dataset.to_csv('clean_dataset_stem_yura.csv',sep=',')